Όπως είδαμε στο μάθημα (στο notebook: https://github.com/datajour-gr/DataJournalism/blob/main/Bachelor%20Lessons%202023/Lesson%206/Classwork6_Scraping_BBC.ipynb), ο κώδικας για να φέρουμε τον τίτλο, το url και την περίληψη όλων των άρθρων από μια σελίδα του bbc, είναι ο εξής:

In [3]:
# εισάγουμε τις βιβλιοθήκες
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# κατεβάζουμε τη σελίδα που θέλουμε
response = requests.get('http://www.bbc.co.uk/news')
doc = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Ξεκινάμε με μια άδεια λίστα
stories_list = []
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    # Φτιάχνουμε ένα λεξικό χωρίς τίποτα
    story_dict = {}
    headline = story.find('h3')
    if headline:
        story_dict['headline'] = headline.text
    link = story.find('a')
    if link:
        story_dict['url'] = link['href']
    summary = story.find('p')
    if summary:
        story_dict['summary'] = summary.text
    # Προσθέτουμε το λεξικό στη λίστα 
    stories_list.append(story_dict)
    
# Δημιουργούμε dataframe από τη λίστα με τα dictionaries
df = pd.DataFrame(stories_list)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Μετατρέπουμε το dataframe σε CSV και το αποθηκεύουμε στο drive.
# Xρησιμοποιούμε το index=False, για να μην αποθηκευτεί ο αύξοντας αριθμός των rows.
df = pd.DataFrame(stories_list)
df.to_csv("/content/drive/MyDrive/bbc.csv", index=False)

In [9]:
df

,headline,url,summary
0,Harry and Meghan in 'near catastrophic car chase',/news/world-us-canada-65625886,A spokesperson says the incident happened afte...
1,Harry's statement blames 'highly aggressive pa...,/news/live/world-us-canada-65620143,NaN
2,World likely to warm beyond key 1.5C limit by ...,/news/science-environment-65602293,The world is likely to hit 1.5C of warming wit...
3,North Carolina bans abortion after 12 weeks,/news/world-us-canada-65615310,Republicans overturn the governor's veto of th...
4,Court orders Elizabeth Holmes to go to prison,/news/world-us-canada-65620371,Holmes was also ordered to pay $452m to the vi...
5,Musk: I will say what I want even if it costs me,/news/technology-65609927,Twitter's owner has been criticised for spread...
6,New negotiators raise hopes of US debt limit deal,/news/world-us-canada-65624823,President Joe Biden has cut short a foreign tr...
7,Break-in at Biden aide's home while agents out...,/news/world-us-canada-65620372,An investigation is launched into how a man en...
8,Watch: Man grabs highly venomous snake on live...,/news/world-australia-65618680,An Australian snake catcher has recorded the m...
9,Sarkozy to wear tag after losing corruption ap...,/news/world-europe-65620064,France's former president was sentenced to thr...


## **Extra tips**

### **Διόρθωση των ελλειπών urls**

Όπως βλέπουμε στο df, κάποια urls δεν είναι ολόκληρα, ενώ κάποια άλλα είναι. Παρατηρούμε ότι το κομμάτι που λείπει είναι το main url, δηλαδή το ```"https://www.bbc.com"```

Μπορούμε να διορθώσουμε τα ελλειπή urls με τον παρακάτω κώδικα:

In [10]:
df['url_new'] = df.apply(lambda x: x['url'] if x['url'].startswith("http") else "https://www.bbc.com" + x['url'], axis=1)

Ο παραπάνω κώδικας δημιουργεί μια νέα στήλη ('url_new') και σε κάθε row βάζει την τιμή του κελιού από τη στήλη 'url' αν η τιμή αρχίζει με 'http', αλλιώς προσθέτει στην αρχή της τιμής του κελιού 'url' το "http://www.bbc.com".

In [11]:
df

,headline,url,summary,url_new
0,Harry and Meghan in 'near catastrophic car chase',/news/world-us-canada-65625886,A spokesperson says the incident happened afte...,https://www.bbc.com/news/world-us-canada-65625886
1,Harry's statement blames 'highly aggressive pa...,/news/live/world-us-canada-65620143,NaN,https://www.bbc.com/news/live/world-us-canada-...
2,World likely to warm beyond key 1.5C limit by ...,/news/science-environment-65602293,The world is likely to hit 1.5C of warming wit...,https://www.bbc.com/news/science-environment-6...
3,North Carolina bans abortion after 12 weeks,/news/world-us-canada-65615310,Republicans overturn the governor's veto of th...,https://www.bbc.com/news/world-us-canada-65615310
4,Court orders Elizabeth Holmes to go to prison,/news/world-us-canada-65620371,Holmes was also ordered to pay $452m to the vi...,https://www.bbc.com/news/world-us-canada-65620371
5,Musk: I will say what I want even if it costs me,/news/technology-65609927,Twitter's owner has been criticised for spread...,https://www.bbc.com/news/technology-65609927
6,New negotiators raise hopes of US debt limit deal,/news/world-us-canada-65624823,President Joe Biden has cut short a foreign tr...,https://www.bbc.com/news/world-us-canada-65624823
7,Break-in at Biden aide's home while agents out...,/news/world-us-canada-65620372,An investigation is launched into how a man en...,https://www.bbc.com/news/world-us-canada-65620372
8,Watch: Man grabs highly venomous snake on live...,/news/world-australia-65618680,An Australian snake catcher has recorded the m...,https://www.bbc.com/news/world-australia-65618680
9,Sarkozy to wear tag after losing corruption ap...,/news/world-europe-65620064,France's former president was sentenced to thr...,https://www.bbc.com/news/world-europe-65620064


# **Άσκηση**

Κάνετε όποια/όποιες από τις παρακάτω ασκήσεις θέλετε:

**1. Βαθμός δυσκολίας: πολύ εύκολο**

Προσαρμόστε κατάλληλα τον κώδικα του notebook https://github.com/datajour-gr/DataJournalism/blob/main/Bachelor%20Lessons%202023/Lesson%206/Scraping%20with%20newspaper3k%20module.ipynb (με το module newspaper3k) για να μαζέψετε δεδομένα από τα άρθρα της κατηγορίας latest του news247 (https://www.news247.gr/latest/) και της efsyn.

**2. Βαθμός δυσκολίας: εύκολο**

Το ίδιο με παραπάνω (1) αλλά μαζέψτε δεδομένα από 3 σελίδες της κατηγορίας latest. 

Hint: χρησιμοποιήστε το /?page=

**3. Βαθμός δυσκολίας: μέτριο**

Προσαρμόστε κατάλληλα τον κώδικα που έχουμε χρησιμοποιήσει με το requests και το BeautifulSoup για να μαζέψετε δεδομένα άρθρων από την κατηγορία των τελευταίων ειδήσεων του news247 (https://www.news247.gr/latest/)

Στη συνέχεια φτιάξτε μια for loop για να μαζέψετε πληροφορίες άρθρων από περισσότερες από μία σελίδες της ίδιας κατηγορίας (χρησιμοποιήστε το /?page=)

**4a. Βαθμός δυσκολίας: προχωρημένο**

Με τη χρήση του requests και του BeautifulSoup:
1. Μαζέψτε (μόνο) τα urls των άρθρων της Κυριακής 14/5/2023 από την κατηγορία των τελευταίων ειδήσεων του news247 (https://www.news247.gr/latest/?page=) και αποθηκεύστε τα σε μία λίστα.
2. Μαζέψτε για κάθε άρθρο στη λίστα τον τίτλο, το url του, την ημερομηνία, τον συντάκτη, την κατηγορία (section) και το κείμενο του άρθρου.

**4b. Βαθμός δυσκολίας: προχωρημένο**

Εναλλακτικά, μπορείτε να μαζέψετε τις πληροφορίες από reviews των χρηστών του trip advisor (π.χ. https://www.tripadvisor.com/AttractionProductReview-g189400-d11476899-Skip_the_Line_Acropolis_of_Athens_Afternoon_Walking_Tour-Athens_Attica.html). 

Μαζέψτε το rating (τα bubbles), τον τίτλο του review (π.χ. Highly recommend tour + museum), το πότε γράφτηκε το review και το text του review.

**4c. Βαθμός δυσκολίας: προχωρημένο**

Μπορείτε να μαζέψετε δεδομένα από σχόλια σε ένα video στο youtube (όνομα χρήστη, ημερομηνία, κείμενο).
